# Imports

In [ ]:
import os
import cv2
import pickle
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from dataset.utils import load_image

In [ ]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.python.keras.applications import VGG19
from tensorflow.python.keras.optimizers import Adam, RMSprop
from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard

# Load Data

In [ ]:
random.seed(2) # To always load the same images in each dataset split

In [ ]:
# Folder containing the datset
data_dir = 'dataset'

In [ ]:
def load_data(filename, data_dir):
    images_path = os.path.join(data_dir, filename)
    with open(images_path, 'rb') as file:
        images = pickle.load(file)
    return images

In [ ]:
# load training data
train_images = load_data('train_images.pkl', data_dir)
train_categories = load_data('train_categories.pkl', data_dir)

In [ ]:
# load validation data
val_images = load_data('val_images.pkl', data_dir)
val_categories = load_data('val_categories.pkl', data_dir)

In [ ]:
# load test data
test_images = load_data('test_images.pkl', data_dir)
test_categories = load_data('test_categories.pkl', data_dir)

In [ ]:
# load mapping
with open('dataset/coco_raw.pickle', 'rb') as file:
    coco_raw = pickle.load(file)
id_category = coco_raw['id_category']

In [ ]:
num_classes = len(category_id)

In [ ]:
num_images_train = len(filenames_train)
num_images_train

### Helper functions to load and display images

In [ ]:
def load_image(path, size=None, grayscale=False):
    """
    Load the image from the given file-path and resize it
    to the given size if not None.
    """

    # Load the image using opencv
    if not grayscale:  # BGR format
        image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    else:  # grayscale format
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    # Resize image if desired.
    if not size is None:
        image = cv2.resize(image, size)

    # Convert image to numpy array and scale pixels so they fall between 0.0 and 1.0
    image = np.array(image) / 255.0

    # Add 1 extra dimension to grayscale images
    if (len(image.shape) == 2):
        image = np.expand_dims(image, axis=-1)

    return image

In [ ]:
def show_image(idx, train):
    """
    Load and plot an image from the training or validation set
    with the given index.
    """

    if train:
        # Use an image from the training-set
        filename = filenames_train[idx]
        categories = categories_train[idx]
    else:
        # Use an image from the validation-set
        filename = filenames_val[idx]
        categories = categories_val[idx]

    # Path for the image-file.
    path = os.path.join(data_dir, filename)

    # Print the captions for this image.
    for category in categories:
        print(category)
    
    # Load the image and plot it.
    img = load_image(path)
    plt.imshow(img)
    plt.show()

### Display a sample image

In [ ]:
show_image(idx=1, train=True)

# Pre-Trained Image Model (VGG16)

In [ ]:
# Download VGG16 model along with the fully-connected layers
model = VGG19(include_top=True, weights='imagenet')
model.summary()

In [ ]:
# Extract the second last layer which is a fully-connected layer
transfer_layer = model.get_layer('block5_pool')

In [ ]:
transfer_layer.output

In [ ]:
# Create the encoder model
conv_model = Model(inputs=model.input, outputs=transfer_layer.output)

In [ ]:
# Start a new Keras Sequential model.
image_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
image_model.add(conv_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
image_model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
image_model.add(Dense(1024, activation='relu'))

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
image_model.add(Dropout(0.5))

# Add the final layer for the actual classification.
image_model.add(Dense(num_classes, activation='sigmoid'))

In [ ]:
optimizer = Adam(lr=1e-3)

In [ ]:
loss = 'categorical_crossentropy'

In [ ]:
metrics = ['categorical_accuracy']

In [ ]:
def print_layer_trainable():
    for layer in conv_model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))

In [ ]:
print_layer_trainable()

In [ ]:
conv_model.trainable = False
for layer in conv_model.layers:
    layer.trainable = False

print_layer_trainable()

In [ ]:
image_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)